# Исследование надежности заемщиков


Во второй части проекта вы выполните шаги 3 и 4. Их вручную проверит ревьюер.
Чтобы вам не пришлось писать код заново для шагов 1 и 2, мы добавили авторские решения в ячейки с кодом. 



## Откройте таблицу и изучите общую информацию о данных

**Задание 1. Импортируйте библиотеку pandas. Считайте данные из csv-файла в датафрейм и сохраните в переменную `data`. Путь к файлу:**

`/datasets/data.csv`

In [ ]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

**Задание 2. Выведите первые 20 строчек датафрейма `data` на экран.**

In [ ]:
data.head(20)

**Задание 3. Выведите основную информацию о датафрейме с помощью метода `info()`.**

In [ ]:
data.info()

## Предобработка данных

### Удаление пропусков

**Задание 4. Выведите количество пропущенных значений для каждого столбца. Используйте комбинацию двух методов.**

In [ ]:
data.isna().sum()

**Задание 5. В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце вы обработаете на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.**

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

**Задание 6. В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработайте значения в этом столбце: замените все отрицательные значения положительными с помощью метода `abs()`.**

In [ ]:
data['days_employed'] = data['days_employed'].abs()

**Задание 7. Для каждого типа занятости выведите медианное значение трудового стажа `days_employed` в днях.**

In [ ]:
data.groupby('income_type')['days_employed'].agg('median')

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Тем более этот столбец не понадобится вам для исследования.

**Задание 8. Выведите перечень уникальных значений столбца `children`.**

In [ ]:
data['children'].unique()

**Задание 9. В столбце `children` есть два аномальных значения. Удалите строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [ ]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**Задание 10. Ещё раз выведите перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [ ]:
data['children'].unique()

### Удаление пропусков (продолжение)

**Задание 11. Заполните пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.**

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**Задание 12. Убедитесь, что все пропуски заполнены. Проверьте себя и ещё раз выведите количество пропущенных значений для каждого столбца с помощью двух методов.**

In [ ]:
data.isna().sum()

### Изменение типов данных

**Задание 13. Замените вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [ ]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

**Задание 14. Обработайте неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведите их к нижнему регистру. Проверьте остальные столбцы.**

In [ ]:
data['education'] = data['education'].str.lower()

**Задание 15. Выведите на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалите их.**

In [ ]:
data.duplicated().sum()

In [ ]:
data = data.drop_duplicates()

### Категоризация данных

**Задание 16. На основании диапазонов, указанных ниже, создайте в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используйте собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [ ]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [ ]:
data['total_income_category'] = data['total_income'].apply(categorize_income)
data['total_income_category'].value_counts()

**Задание 17. Выведите на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [ ]:
data['purpose'].unique()

**Задание 18. Создайте функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Используйте собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучите данные в столбце `purpose` и определите, какие подстроки помогут вам правильно определить категорию.**

In [ ]:
def categorize_purpose(purpose):
    try:
        if 'автом' in purpose:
            return 'операции с автомобилем'
        elif 'жил' in purpose or 'недвиж' in purpose:
            return 'операции с недвижимостью'
        elif 'свад' in purpose:
            return 'проведение свадьбы'
        elif 'образов' in purpose:
            return 'получение образования'
    except:
        return 'нет категории'

In [ ]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследуйте данные и ответьте на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [ ]:
def children_determine(children):
    if children > 0: return 1
    else: return 0

# добавим новый столбец с двойным признаком в массив
data['child_exist'] = data['children'].apply(children_determine)

data_pivot = data.pivot_table(index = ['child_exist'], values = 'debt').round(3)
#data_pivot['ratio'] = data_pivot[1] / data_pivot[0]
data_pivot.head()

**Вывод**: Наличие детей увеличивает вероятность задолженности по сравнению с отсутствием детей у заемщика. Семьи с большим количеством детей чаще сталкиваются с финансовыми пробемами и имеют сложности с возвратом кредитов банку.

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
data_pivot = data.pivot_table(index = ['family_status'], columns = 'debt',values ='income_type', aggfunc = 'count')

# посчитаем вероятность задолженности для каждого вида семейного положения
data_pivot['ratio'] = round(data_pivot[1] / (data_pivot[0] + data_pivot[1]), 3)
data_pivot.sort_values('ratio', ascending = False)

**Вывод**:  В категории  "не женат/не замужем" среди заемщиков увеличена вероятность образования задолженности перед банком. Самая дисциплинированная группа заемщиков -  в статусе "вдова/вдовец". Возможно, необходимо сгруппировать данные по двум категориям "Есть партнерские отношения" и "Нет партнерских отношений"для выявления более точной картины.


#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
data_pivot = data.pivot_table(index = ['total_income_category'], columns = 'debt', values = 'gender', aggfunc = 'count')

# посчитаем вероятность задолженности для каждой группы доходов

data_pivot['ratio'] = round(data_pivot[1] / (data_pivot[0] + data_pivot[1]), 3)
data_pivot.sort_values('ratio', ascending = False)


**Вывод:** 
Получившиеся вероятности задолженности, отсортированные по убыванию:
доход в диапазоне от 0 до 30 000 рублей в месяц (категрия Е);
доход в диапазоне от 50 001 до 200 000 рублей рублей (категория С);
доход  от 1 000 001 рублей и выше (категория А);
доход больше 195,8 тыс. рублей (категория В);
доход от 30 001 до 50 000 рублей( категория D).

Самая дисциплинированная группа заемщиков - это клиенты с доходом от  диапазоне от 30 001 до 50 000 рублей ( категория D), затем идут клиенты с доходом от 200 001 до 1 000 000 рублей( категория В), клиенты с доходом  от 1 000 001 рублей и выше (категория А), затем клиенты с доходом от от 50 001 до 200 000 рублей ( категория С) и те, кто зарабатывают от 0 до 30 000 рублей. Самые не дисциплинированные клиенты с доходом до 30 000 рублей. Зависимости между уровнем дохода и возвратом кредита в срок нет.

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [ ]:
# Ваш код будет здесь. Вы можете создавать новые ячейки.
data_pivot = data.pivot_table(index = ['purpose_category'], columns = 'debt', values = 'gender', aggfunc = 'count')

# посчитаем вероятность задолженности для каждой группы доходов
data_pivot['ratio'] = round(data_pivot[1] / (data_pivot[0] + data_pivot[1]), 3)
data_pivot.sort_values('ratio', ascending = False)

**Вывод:** Получившиеся вероятности задолженности для каждой цели получения кредита, отсортированные по убыванию:

операции с автомобилем;
получение образования;
проведение свадьбы;
операции с недвижимостью.
Самая низкая вероятность задолженности у клиентов, которые берут кредит для операций с недвижимостью. Хуже всех платят по кредитам  на операции с автомобилями и для получения образования.

#### 3.5 Приведите возможные причины появления пропусков в исходных данных.

*Ответ:* В данных часто бывают пропуски по причинам влияния человеческого фактора (неправильного ввода данных, сокрытия полной информации), технического сбоя и т.д.Существует три основных типа пропущенных данных:

Пропал совершенно случайно (MCAR);
Случайно пропал (MAR);
Не пропал случайно (NMAR).

#### 3.6 Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:* 
Среднее значение может быть не совсем объективным отражением данных в данном случае, так как на него могут влиять выбросы (очень большие или очень маленькие значения в наборе данных). Медиана показывает середину наших данных. Медиана считается как значение, расположенное по середине ряда отсортированных значений. Медиана лишена недостатков среднего значения, на нее не влияют выбросы.



### Шаг 4: общий вывод.

Напишите ваш общий вывод.

Цель проекта: "Исследование надежности заемщиков".

**Этапы проведения исследования**:

**1 этап:** Изучение общей информации о данных.
Входные данные для исследования — статистика о 21 525 клиентах банка (путь к файлу: /datasets/data.csv). Каждый клиент описывается 12 признаками, 4 из которых относятся к количественным признакам, 8 – к категориальным.При подготовке исследования использовались следующие основные группы показателей: уровень дохода, семейное положение,количество детей, цели кредита и их влияние на факт погашения кредита в срок.

**2 этап:** Предобработка данных.
2.1.Выявили пропущенные значения по общему трудовому стажу ('days_employed') и ежемесячному доходу ('total_income') у 2174 клиентов, это около 10% от общего числа клиентов. Необходимо выявить причину подобных пропусков. 

2.2.Обработали аномальные значения:заменили все отрицательные значения положительными в столбце "days_employed" и некорректные данные в столбце "children". 

2.3.Удалили пропуски: на сумму дохода сильнее всего влияет тип занятости, поэтому пропуски в этом столбце заполнены  медианным значением по каждому типу из столбца "income_type". 

2.4.Изменили типы данных: заменили  тип данных в столбце "total_income" на целочисленный. 

2.5.Обработали  неявные дубликаты в столбце "education". 

2.6.Провели категоризацию данных для последующей обработки данных.Созданы два вида категорий: один — по доходу('total_income_category'), другой — по целям ('purpose_category').

**3 этап:** Ответы на вопросы и выводы.

Перед проведением исследования было поставлено четыре вопроса/гипотезы:

1.Количество детей влияет на возврат кредита в срок.
2.Возврат кредита в срок зависит от семейного положения.
3.Уровень дохода влияет на возврат кредита в срок.
4.Цели кредита влияют на его возврат в срок.

В результате исследования выявлены следующие зависисмости:
1. Гипотеза подтверждена.
Кредитополучатели без детей являются более дисциплинированными, чем клиенты, у которых есть дети. 
Наличие ребенка накладывает на клиента дополнительные обязательства и клиенты либо менее ответственно относятся к своим кредитным обязательствам, либо не имеют возможности в срок оплачивать обязательства.Самые надежные заемщики с самой низкой долей просрочек по кредиту -  заемщики без детей.
2. Гипотеза  подтверждена.
В категории "не женат/не замужем" среди заемщиков увеличена вероятность образования задолженности перед банком. Самая дисциплинированная группа заемщиков - в статусе "вдова/вдовец". Возможно, необходимо сгруппировать данные по двум категориям "Есть партнерские отношения" и "Нет партнерских отношений"для выявления более точной картины.
Можно расширить вывод о том, что мало не иметь детей, чтобы являться надежным заемщиком для банка, надо еще и не быть одиноким (быть женатым / замужем).
3. Гипотеза подтерждена.
Самая дисциплинированная группа заемщиков - это клиенты с доходом от диапазоне от 30 001 до 50 000 рублей ( категория D), затем идут клиенты с доходом от 200 001 до 1 000 000 рублей( категория В), клиенты с доходом от 1 000 001 рублей и выше (категория А), затем клиенты с доходом от от 50 001 до 200 000 рублей ( категория С) и те, кто зарабатывают от 0 до 30 000 рублей. Самые не дисциплинированные клиенты с доходом до 30 000 рублей. 
Самая высокая доля просрочек по кредиту (0.091) у потенциальных заемщиков из категории 'Е' с уровнем дохода 0-30000 руб.
4. Гипотеза подтерждена.
Получившиеся вероятности задолженности для каждой цели получения кредита, отсортированные по убыванию:
операции с автомобилем; получение образования; проведение свадьбы; операции с недвижимостью. Самая низкая вероятность задолженности у клиентов, которые берут кредит для операций с недвижимостью. Хуже всех платят по кредитам на операции с автомобилями и для получения образования.

**Портрет условного идеального клиента банка:** 

Результаты исследования факторов скоринга банков для определения надежности заемщиков показали, что идеальные заемщики для банка при рассмотрении заявки на кредиты - это:

желающие взять кредит для покупки недвижимости;
семейные люди без детей;
с уровнем дохода от 30 001 до 50 000 руб.
